<a href="https://colab.research.google.com/github/merajali384022/mehraj-ali/blob/main/AI_STUDY_PAL_(MAJOR_PROJECT).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
# ai_study_pal_gradio_full.py
# 🎓 AI Study Pal - Full Gradio Web App (10 subjects x 20 MCQs)
# ---------------------------------------------------------------
import random
import io
import os
import pandas as pd
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import gradio as gr
import tempfile

# First-time NLTK downloads (safe to run repeatedly)
nltk.download('punkt')
nltk.download('stopwords')

# =========================
# QUIZ DATABASE (10 subjects × 20 questions each)
# (Copied from user's full dataset)
# =========================
quizzes = {
    "English": [
        ("What is a synonym of 'happy'?", ["Sad", "Joyful", "Angry", "Tired"], "Joyful"),
        ("Which is a noun?", ["Run", "Beautiful", "Apple", "Quickly"], "Apple"),
        ("Choose the correct past tense: 'He ___ to school yesterday.'", ["go", "goes", "went", "gone"], "went"),
        ("What is the opposite of 'ancient'?", ["Old", "Modern", "Past", "Olden"], "Modern"),
        ("Find the adjective: 'The tall building looks strong.'", ["tall", "building", "looks", "strong"], "tall"),
        ("Which word is a verb?", ["Dance", "Music", "Chair", "Book"], "Dance"),
        ("Choose the correct article: 'I saw ___ elephant.'", ["a", "an", "the", "no article"], "an"),
        ("Which word is a pronoun?", ["She", "Car", "Play", "Beautiful"], "She"),
        ("What is a conjunction in 'Ravi and Riya are friends'?", ["Ravi", "and", "Riya", "friends"], "and"),
        ("Which sentence is correct?", ["He go school.", "He goes to school.", "He going school.", "He gone school."], "He goes to school."),
        ("What is an antonym of 'strong'?", ["Weak", "Brave", "Powerful", "Bold"], "Weak"),
        ("Identify the adverb: 'She runs quickly.'", ["She", "runs", "quickly", "None"], "quickly"),
        ("Choose the plural of 'child'.", ["childs", "childes", "children", "childen"], "children"),
        ("What is the past form of 'write'?", ["writed", "wrote", "written", "writes"], "wrote"),
        ("Find the correct preposition: 'The cat is ___ the table.'", ["in", "on", "under", "by"], "under"),
        ("Which of these is a question word?", ["Who", "Dog", "Run", "Book"], "Who"),
        ("Choose the correct spelling:", ["Enviroment", "Environment", "Environmant", "Environmnet"], "Environment"),
        ("Identify the correct sentence:", ["I am go school", "I am going to school", "I going school", "I gone to school"], "I am going to school"),
        ("What is the opposite of 'begin'?", ["Start", "End", "Move", "Go"], "End"),
        ("Find the verb: 'Birds fly in the sky.'", ["Birds", "fly", "sky", "the"], "fly"),
    ],
    "Mathematics": [
        ("What is 12 × 8?", ["96", "88", "108", "86"], "96"),
        ("Simplify: 45 ÷ 9 =", ["4", "5", "6", "7"], "5"),
        ("The value of π (pi) is approximately:", ["2.14", "3.14", "4.13", "3.41"], "3.14"),
        ("Find the square of 15:", ["125", "200", "225", "250"], "225"),
        ("What is 25% of 200?", ["40", "50", "25", "60"], "50"),
        ("Convert 0.75 to fraction:", ["3/4", "1/2", "2/3", "1/3"], "3/4"),
        ("Simplify: 3² + 4² =", ["7", "25", "12", "9"], "25"),
        ("What is 1 km in meters?", ["10", "100", "1000", "10000"], "1000"),
        ("The smallest prime number is:", ["0", "1", "2", "3"], "2"),
        ("Find the perimeter of a square with side 8 cm:", ["16", "24", "32", "64"], "32"),
        ("What is 7 × 6 + 4?", ["42", "46", "38", "50"], "46"),
        ("Simplify: (20 – 5) × 2 =", ["10", "30", "25", "35"], "30"),
        ("The LCM of 4 and 6 is:", ["12", "8", "10", "14"], "12"),
        ("The HCF of 24 and 36 is:", ["4", "6", "12", "8"], "12"),
        ("What is the cube of 3?", ["6", "9", "27", "18"], "27"),
        ("If 5x = 25, find x:", ["10", "5", "25", "50"], "5"),
        ("Find the mean of 2, 4, 6, 8:", ["4", "5", "6", "7"], "5"),
        ("What is 1000 – 675?", ["225", "325", "335", "300"], "325"),
        ("Find the next term: 2, 4, 8, 16, __", ["18", "24", "32", "64"], "32"),
        ("What is the value of √49?", ["6", "7", "8", "9"], "7"),
    ],
    "Science": [
        ("What is the chemical symbol for water?", ["H2O", "O2", "CO2", "H2"], "H2O"),
        ("Which gas is used by plants during photosynthesis?", ["Oxygen", "Nitrogen", "Carbon dioxide", "Helium"], "Carbon dioxide"),
        ("What planet is known as the Red Planet?", ["Mars", "Venus", "Earth", "Jupiter"], "Mars"),
        ("Which part of the plant conducts photosynthesis?", ["Root", "Leaf", "Stem", "Flower"], "Leaf"),
        ("What is the speed of light?", ["3×10^8 m/s", "3000 m/s", "300 km/h", "30 m/s"], "3×10^8 m/s"),
        ("What is the boiling point of water?", ["50°C", "100°C", "150°C", "200°C"], "100°C"),
        ("Which organ pumps blood?", ["Brain", "Lungs", "Heart", "Kidney"], "Heart"),
        ("Which gas do humans exhale?", ["Oxygen", "Carbon dioxide", "Nitrogen", "Helium"], "Carbon dioxide"),
        ("Energy cannot be created or destroyed — this is:", ["Law of Motion", "Law of Conservation of Energy", "Boyle’s Law", "Ohm’s Law"], "Law of Conservation of Energy"),
        ("What is the unit of force?", ["Joule", "Newton", "Pascal", "Volt"], "Newton"),
        ("Which vitamin is produced by sunlight?", ["A", "B", "C", "D"], "D"),
        ("Which instrument measures temperature?", ["Barometer", "Thermometer", "Hygrometer", "Altimeter"], "Thermometer"),
        ("What is Earth’s natural satellite?", ["Sun", "Mars", "Moon", "Venus"], "Moon"),
        ("What part of the atom has a positive charge?", ["Electron", "Proton", "Neutron", "Nucleus"], "Proton"),
        ("Which blood cells fight infections?", ["RBCs", "WBCs", "Platelets", "Plasma"], "WBCs"),
        ("Which device converts electrical energy to light?", ["Bulb", "Battery", "Motor", "Generator"], "Bulb"),
        ("What is a change from liquid to gas called?", ["Condensation", "Evaporation", "Freezing", "Melting"], "Evaporation"),
        ("Which planet has rings?", ["Earth", "Mars", "Saturn", "Mercury"], "Saturn"),
        ("Which is the largest organ of the human body?", ["Heart", "Skin", "Brain", "Lungs"], "Skin"),
        ("Which metal is liquid at room temperature?", ["Mercury", "Iron", "Aluminium", "Gold"], "Mercury"),
    ],
    "Biology": [
        ("What is the basic unit of life?", ["Cell", "Atom", "Tissue", "Organ"], "Cell"),
        ("Plants make their food through:", ["Digestion", "Photosynthesis", "Respiration", "Transpiration"], "Photosynthesis"),
        ("Which organelle is called the powerhouse?", ["Nucleus", "Mitochondria", "Chloroplast", "Ribosome"], "Mitochondria"),
        ("What is the human body's largest organ?", ["Heart", "Skin", "Liver", "Lungs"], "Skin"),
        ("Which part of the cell contains genetic material?", ["Nucleus", "Cytoplasm", "Cell wall", "Membrane"], "Nucleus"),
        ("What transports oxygen in blood?", ["RBCs", "WBCs", "Platelets", "Plasma"], "RBCs"),
        ("Which system controls body activities?", ["Nervous system", "Digestive system", "Skeletal system", "Circulatory system"], "Nervous system"),
        ("Which vitamin helps in blood clotting?", ["A", "C", "K", "D"], "K"),
        ("What is the green pigment in plants?", ["Chlorophyll", "Haemoglobin", "Melanin", "Carotene"], "Chlorophyll"),
        ("Which process removes waste from the body?", ["Respiration", "Excretion", "Circulation", "Digestion"], "Excretion"),
        ("Which organ purifies blood?", ["Heart", "Lungs", "Kidneys", "Brain"], "Kidneys"),
        ("Which disease is caused by mosquitoes?", ["Tuberculosis", "Malaria", "Typhoid", "Cholera"], "Malaria"),
        ("Which part of brain controls balance?", ["Cerebrum", "Cerebellum", "Medulla", "Pons"], "Cerebellum"),
        ("What carries genetic information?", ["DNA", "Protein", "Lipid", "RNA"], "DNA"),
        ("Which blood group is universal donor?", ["A", "B", "O", "AB"], "O"),
        ("Which gas do humans need to live?", ["Carbon dioxide", "Oxygen", "Nitrogen", "Hydrogen"], "Oxygen"),
        ("How many bones in an adult human body?", ["206", "205", "210", "208"], "206"),
        ("Which kingdom includes bacteria?", ["Plantae", "Animalia", "Monera", "Fungi"], "Monera"),
        ("Which part of flower produces pollen?", ["Stamen", "Pistil", "Petal", "Sepal"], "Stamen"),
        ("Which disease affects lungs?", ["Asthma", "Diabetes", "Arthritis", "Jaundice"], "Asthma"),
    ],
    "Chemistry": [
        ("What is the symbol for sodium?", ["So", "S", "Na", "Sn"], "Na"),
        ("Water is made of:", ["Hydrogen and Oxygen", "Oxygen and Carbon", "Hydrogen only", "Carbon dioxide"], "Hydrogen and Oxygen"),
        ("HCl is known as:", ["Sulphuric acid", "Hydrochloric acid", "Nitric acid", "Carbonic acid"], "Hydrochloric acid"),
        ("What is the atomic number of Oxygen?", ["6", "7", "8", "9"], "8"),
        ("Which gas is used in balloons?", ["Oxygen", "Nitrogen", "Helium", "Hydrogen"], "Helium"),
        ("The pH of pure water is:", ["1", "5", "7", "9"], "7"),
        ("Which element is liquid at room temperature?", ["Gold", "Mercury", "Iron", "Aluminium"], "Mercury"),
        ("NaCl is commonly known as:", ["Sugar", "Salt", "Soda", "Acid"], "Salt"),
        ("Which acid is found in lemon?", ["Sulphuric acid", "Acetic acid", "Citric acid", "Nitric acid"], "Citric acid"),
        ("What is a chemical symbol for carbon?", ["C", "Ca", "Co", "Cu"], "C"),
        ("Which element is most abundant in air?", ["Oxygen", "Nitrogen", "Carbon dioxide", "Hydrogen"], "Nitrogen"),
        ("Which metal reacts with water?", ["Copper", "Gold", "Sodium", "Silver"], "Sodium"),
        ("CO2 turns limewater:", ["Red", "Milky", "Blue", "Green"], "Milky"),
        ("Rusting of iron needs:", ["Water and oxygen", "Nitrogen", "Carbon dioxide", "Heat"], "Water and oxygen"),
        ("Which gas supports combustion?", ["Nitrogen", "Oxygen", "Helium", "Hydrogen"], "Oxygen"),
        ("Which particle has no charge?", ["Proton", "Electron", "Neutron", "Ion"], "Neutron"),
        ("Atomic number represents:", ["Neutrons", "Electrons", "Protons", "Mass"], "Protons"),
        ("H2SO4 is:", ["Base", "Acid", "Salt", "Metal"], "Acid"),
        ("What is the formula of methane?", ["CH4", "CO2", "H2O", "NH3"], "CH4"),
        ("What type of change is burning paper?", ["Physical", "Chemical", "Temporary", "Reversible"], "Chemical"),
    ],
    "Physics": [
        ("Unit of force is:", ["Joule", "Watt", "Newton", "Pascal"], "Newton"),
        ("Speed = ?", ["Distance × Time", "Distance / Time", "Time / Distance", "Force × Mass"], "Distance / Time"),
        ("Unit of energy is:", ["Newton", "Joule", "Watt", "Meter"], "Joule"),
        ("Who discovered gravity?", ["Newton", "Einstein", "Galileo", "Faraday"], "Newton"),
        ("SI unit of power:", ["Joule", "Watt", "Volt", "Newton"], "Watt"),
        ("Which instrument measures electric current?", ["Voltmeter", "Ammeter", "Thermometer", "Barometer"], "Ammeter"),
        ("Which lens converges light?", ["Concave", "Convex", "Plain", "Flat"], "Convex"),
        ("Which mirror is used in rear-view mirrors?", ["Concave", "Plane", "Convex", "Spherical"], "Convex"),
        ("Sound travels fastest in:", ["Air", "Water", "Steel", "Vacuum"], "Steel"),
        ("1 horsepower equals:", ["746 W", "500 W", "1000 W", "10 W"], "746 W"),
        ("Who gave the three laws of motion?", ["Einstein", "Newton", "Faraday", "Bohr"], "Newton"),
        ("Energy stored in a stretched string:", ["Kinetic", "Potential", "Heat", "Light"], "Potential"),
        ("SI unit of frequency:", ["Hertz", "Ohm", "Volt", "Tesla"], "Hertz"),
        ("SI unit of charge:", ["Coulomb", "Ampere", "Volt", "Farad"], "Coulomb"),
        ("Which law defines force?", ["Ohm’s law", "Newton’s Second Law", "Boyle’s Law", "Hooke’s Law"], "Newton’s Second Law"),
        ("Light year measures:", ["Time", "Distance", "Speed", "Mass"], "Distance"),
        ("Which color bends most in rainbow?", ["Red", "Violet", "Blue", "Green"], "Violet"),
        ("Energy cannot be destroyed — called?", ["Conservation of Energy", "Law of Gravity", "Newton’s Third Law", "Inertia"], "Conservation of Energy"),
        ("Which current flows in one direction only?", ["AC", "DC", "Magnetic", "Static"], "DC"),
        ("Which planet exerts strongest gravity?", ["Earth", "Jupiter", "Mars", "Venus"], "Jupiter"),
    ],
    "Geography": [
        ("What is the largest ocean?", ["Atlantic", "Indian", "Pacific", "Arctic"], "Pacific"),
        ("Which is the longest river?", ["Nile", "Amazon", "Yangtze", "Ganga"], "Nile"),
        ("Which line divides Earth equally?", ["Tropic of Cancer", "Equator", "Prime Meridian", "Arctic Circle"], "Equator"),
        ("Which planet is known as the blue planet?", ["Earth", "Mars", "Venus", "Jupiter"], "Earth"),
        ("Mount Everest is located in:", ["India", "China", "Nepal", "Bhutan"], "Nepal"),
        ("Which is a renewable resource?", ["Coal", "Oil", "Solar", "Gas"], "Solar"),
        ("What causes day and night?", ["Earth’s rotation", "Earth’s revolution", "Sun’s rotation", "Moon’s orbit"], "Earth’s rotation"),
        ("The Sahara Desert is in:", ["Asia", "Africa", "Australia", "America"], "Africa"),
        ("What is a group of islands called?", ["Delta", "Archipelago", "Peninsula", "Isthmus"], "Archipelago"),
        ("What is India’s capital?", ["Mumbai", "Delhi", "Kolkata", "Chennai"], "Delhi"),
        ("Which layer of Earth has magma?", ["Crust", "Mantle", "Core", "Atmosphere"], "Mantle"),
        ("What is the study of weather?", ["Astronomy", "Geology", "Meteorology", "Biology"], "Meteorology"),
        ("Which gas causes greenhouse effect?", ["CO2", "O2", "N2", "H2"], "CO2"),
        ("Which continent is largest?", ["Asia", "Africa", "Europe", "Antarctica"], "Asia"),
        ("What causes tides?", ["Sun", "Wind", "Moon", "Rain"], "Moon"),
        ("Which is a man-made resource?", ["Water", "Coal", "Plastic", "Forest"], "Plastic"),
        ("Which ocean is smallest?", ["Indian", "Arctic", "Pacific", "Atlantic"], "Arctic"),
        ("Which country is called 'Land of Rising Sun'?", ["China", "Japan", "Korea", "Thailand"], "Japan"),
        ("Which map shows boundaries?", ["Political map", "Physical map", "Climate map", "Resource map"], "Political map"),
        ("Which ocean is between Africa and Australia?", ["Pacific", "Indian", "Arctic", "Atlantic"], "Indian"),
    ],
    "History": [
        ("Who discovered India by sea?", ["Columbus", "Vasco da Gama", "Magellan", "Cook"], "Vasco da Gama"),
        ("Who was the first President of India?", ["Gandhi", "Rajendra Prasad", "Nehru", "Patel"], "Rajendra Prasad"),
        ("When did India get independence?", ["1945", "1947", "1950", "1939"], "1947"),
        ("Who built the Taj Mahal?", ["Akbar", "Shah Jahan", "Aurangzeb", "Humayun"], "Shah Jahan"),
        ("The Great Wall is in:", ["India", "China", "Japan", "Egypt"], "China"),
        ("Where did Buddha attain enlightenment?", ["Sarnath", "Bodh Gaya", "Lumbini", "Kushinagar"], "Bodh Gaya"),
        ("Who started the Non-Cooperation Movement?", ["Gandhi", "Nehru", "Tilak", "Bose"], "Gandhi"),
        ("World War I began in:", ["1914", "1939", "1918", "1945"], "1914"),
        ("Who founded the Maurya Empire?", ["Ashoka", "Bindusara", "Chandragupta Maurya", "Harsha"], "Chandragupta Maurya"),
        ("Which year did World War II end?", ["1939", "1945", "1918", "1965"], "1945"),
        ("Who was known as 'Iron Man of India'?", ["Nehru", "Patel", "Ambedkar", "Gandhi"], "Patel"),
        ("Which empire built Red Fort?", ["Mughal", "British", "Gupta", "Maurya"], "Mughal"),
        ("Who wrote Indian Constitution?", ["Nehru", "Ambedkar", "Gandhi", "Patel"], "Ambedkar"),
        ("Which year did French Revolution start?", ["1776", "1789", "1815", "1848"], "1789"),
        ("Where is Indus Valley Civilization located?", ["Egypt", "India-Pakistan", "China", "Iran"], "India-Pakistan"),
        ("Who discovered America?", ["Columbus", "Vasco da Gama", "Magellan", "Cook"], "Columbus"),
        ("Who was the first Emperor of China?", ["Qin Shi Huang", "Kublai Khan", "Sun Yat-sen", "Han"], "Qin Shi Huang"),
        ("Who invented the printing press?", ["Newton", "Gutenberg", "Edison", "Bell"], "Gutenberg"),
        ("When did World War II start?", ["1939", "1914", "1945", "1950"], "1939"),
        ("Who was India’s first Prime Minister?", ["Rajendra Prasad", "Gandhi", "Nehru", "Patel"], "Nehru"),
    ],
    "Commerce": [
        ("What is the main goal of business?", ["Profit", "Loss", "Donation", "Competition"], "Profit"),
        ("Money is a:", ["Barter", "Medium of exchange", "Good", "Service"], "Medium of exchange"),
        ("Which is a current asset?", ["Land", "Building", "Stock", "Machine"], "Stock"),
        ("What is 'capital'?", ["Workers", "Money invested", "Customers", "Product"], "Money invested"),
        ("What is the meaning of GDP?", ["Gross Domestic Product", "Gross Deposit Profit", "Government Data Policy", "General Daily Production"], "Gross Domestic Product"),
        ("What is 'banking'?", ["Production", "Trade", "Financial service", "Marketing"], "Financial service"),
        ("Which tax is indirect?", ["Income tax", "GST", "Property tax", "Corporate tax"], "GST"),
        ("Which book records daily transactions?", ["Journal", "Ledger", "Invoice", "Bill"], "Journal"),
        ("Which is an example of intangible good?", ["Car", "Service", "Table", "Book"], "Service"),
        ("Which term means selling goods abroad?", ["Import", "Export", "Exchange", "Barter"], "Export"),
        ("What does 'budget' mean?", ["Spending plan", "Bill", "Profit", "Tax"], "Spending plan"),
        ("Which is a liability?", ["Loan", "Stock", "Furniture", "Building"], "Loan"),
        ("Which document proves ownership?", ["Bill", "Receipt", "Certificate", "Invoice"], "Certificate"),
        ("What is 'marketing'?", ["Buying", "Selling", "Promoting and selling products", "Accounting"], "Promoting and selling products"),
        ("Which department handles money?", ["Finance", "HR", "Sales", "Production"], "Finance"),
        ("Who manages a company?", ["Manager", "Board of Directors", "Worker", "Shareholder"], "Board of Directors"),
        ("Which business is owned by one person?", ["Partnership", "Company", "Sole Proprietorship", "Cooperative"], "Sole Proprietorship"),
        ("What is 'revenue'?", ["Profit", "Income before expenses", "Loss", "Debt"], "Income before expenses"),
        ("What is 'inventory'?", ["Machines", "Stock of goods", "Office furniture", "Building"], "Stock of goods"),
        ("Which sector includes farming?", ["Primary", "Secondary", "Tertiary", "Service"], "Primary"),
    ],
    "Social Studies": [
        ("Which body governs a country?", ["Government", "School", "Factory", "Hospital"], "Government"),
        ("Democracy means:", ["Rule by one", "Rule by people", "Rule by army", "Rule by rich"], "Rule by people"),
        ("Which revolution began in 1789?", ["Industrial", "French", "American", "Russian"], "French"),
        ("Who wrote Indian Constitution?", ["Gandhi", "Ambedkar", "Nehru", "Patel"], "Ambedkar"),
        ("What is GDP?", ["Gross Domestic Product", "Government Data Policy", "General Daily Plan", "Growth Domestic Percent"], "Gross Domestic Product"),
        ("What is the main occupation in India?", ["Farming", "Mining", "IT", "Manufacturing"], "Farming"),
        ("Who was India’s first Prime Minister?", ["Gandhi", "Patel", "Nehru", "Ambedkar"], "Nehru"),
        ("Which is the smallest continent?", ["Asia", "Europe", "Australia", "Africa"], "Australia"),
        ("Which organization promotes peace?", ["UN", "WHO", "IMF", "WTO"], "UN"),
        ("When did India become republic?", ["1947", "1949", "1950", "1962"], "1950"),
        ("Who discovered sea route to India?", ["Columbus", "Vasco da Gama", "Magellan", "Cook"], "Vasco da Gama"),
        ("What is capital of India?", ["Delhi", "Mumbai", "Kolkata", "Chennai"], "Delhi"),
        ("Which is a primary sector?", ["Agriculture", "Banking", "IT", "Education"], "Agriculture"),
        ("Who elects the President of India?", ["PM", "Parliament", "People", "Judges"], "Parliament"),
        ("Which country is India’s neighbor?", ["USA", "France", "China", "Brazil"], "China"),
        ("Which is the largest democracy?", ["USA", "India", "UK", "Japan"], "India"),
        ("Who was known as Father of Nation?", ["Ambedkar", "Gandhi", "Nehru", "Patel"], "Gandhi"),
        ("Which festival celebrates harvest?", ["Diwali", "Holi", "Pongal", "Eid"], "Pongal"),
        ("Who enforces law and order?", ["Police", "Army", "Court", "Teacher"], "Police"),
        ("Which day is celebrated as Independence Day?", ["Jan 26", "Aug 15", "Oct 2", "Dec 25"], "Aug 15"),
    ],
}

# =========================
# RESOURCES (Wikipedia links)
# =========================
resources = {
    "English": [
        "https://en.wikipedia.org/wiki/English_language",
        "https://www.khanacademy.org/humanities/grammar",
        "https://www.khanglobalstudies.com/explore/khan-sir-courses",
        "https://library.sjsu.edu/openresources/greek",
        "https://www.textkit.com/t/ancient-greek-resources/21091"
    ],
    "Mathematics": [
        "https://en.wikipedia.org/wiki/Mathematics",
        "https://www.khanacademy.org/math"
    ],
    "Science": [
        "https://en.wikipedia.org/wiki/Science",
        "https://www.ck12.org/science/",
        "https://www.khanglobalstudies.com/explore/khan-sir-courses"
    ],
    "Biology": [
        "https://en.wikipedia.org/wiki/Biology",
        "https://www.khanglobalstudies.com/explore/khan-sir-courses?srsltid=",
        "https://www.khanacademy.org/science/biology"
    ],
    "Chemistry": [
        "https://en.wikipedia.org/wiki/Chemistry",
        "https://www.khanacademy.com/science/chemistry",
        "https://www.khanglobalstudies.com/explore/khan-sir-courses?srsltid="
    ],
    "Physics": [
        "https://en.wikipedia.org/wiki/Physics",
        "https://www.khanacademy.org/science/physics",
        "https://www.khanglobalstudies.com/explore/khan-sir-courses?srsltid="
    ],
    "Geography": [
        "https://en.wikipedia.org/wiki/Geography",
        "https://education.nationalgeographic.org/resource/atlas-of-world-geography/",
        "https://www.khanglobalstudies.com/explore/khan-sir-courses?srsltid=",
        "https://clcl.uiowa.edu/language-resources/greek-language-and-culture-resources"
    ],
    "History": [
        "https://en.wikipedia.org/wiki/History",
        "https://www.khanglobalstudies.com/explore/khan-sir-courses?srsltid=",
        "https://education.nationalgeographic.org/resource/resource-library-ancient-greece/",
        "https://www.textkit.com/t/ancient-greek-resources/21091"
    ],
    "Commerce": [
        "https://en.wikipedia.org/wiki/Commerce",
        "https://www.investopedia.com/terms/c/commerce.asp",
        "https://www.khanglobalstudies.com/courses/ugc-net-jrf-commerce-june-2025?srsltid=",
        "https://www.khanglobalstudies.com/explore/khan-sir-courses?srsltid="
    ],
    "Social Studies": [
        "https://en.wikipedia.org/wiki/Social_studies",
        "https://www.khanglobalstudies.com/explore/khan-sir-courses?srsltid=",
        "https://www.khanacademy.org/humanities/civics-and-government",
        "https://library.sjsu.edu/openresources/greek"
    ],
    "Khan Academy": "https://www.khanacademy.org/",
    "GeeksforGeeks": "https://www.geeksforgeeks.org/"
}

# =========================
# HELPERS: feedback, study tips, plan, quotes
# =========================

motivational_quotes = [
    "The only way to do great work is to love what you do.",
    "Believe you can and you're halfway there.",
    "The future belongs to those who believe in the beauty of their dreams.",
    "Don't watch the clock; do what it does. Keep going.",
    "The mind is not a vessel to be filled, but a fire to be kindled.",
    "Strive not to be a success, but rather to be of value.",
    "Your time is limited, don't waste it living someone else's life.",
    "The best way to predict the future is to create it.",
    "The only limit to our realization of tomorrow will be our doubts of today.",
    "Learning is a treasure that will follow its owner everywhere."
]

def get_random_quote():
    return random.choice(motivational_quotes)

# General study tips for each subject
subject_study_tips = {
    "English": [
        "• Read widely to improve vocabulary and comprehension.",
        "• Practice writing regularly, focusing on grammar and structure.",
        "• Learn and use new words in sentences.",
        "• Pay attention to punctuation.",
        "• Review different literary devices."
    ],
    "Mathematics": [
        "• Practice problems daily.",
        "• Understand the concepts before memorizing formulas.",
        "• Review your mistakes to learn from them.",
        "• Try to solve problems in different ways.",
        "• Don't be afraid to ask for help."
    ],
    "Science": [
        "• Understand the fundamental principles.",
        "• Use diagrams and visual aids.",
        "• Conduct experiments if possible.",
        "• Relate concepts to real-world examples.",
        "• Review scientific terms and definitions."
    ],
    "Biology": [
        "• Focus on understanding biological processes.",
        "• Use flashcards for memorizing terms and definitions.",
        "• Draw diagrams of systems and cycles.",
        "• Study with a partner to discuss concepts.",
        "• Review classification and taxonomy."
    ],
    "Chemistry": [
        "• Understand the periodic table.",
        "• Practice balancing chemical equations.",
        "• Learn about different types of chemical reactions.",
        "•   Review the properties of elements and compounds.",
        "•   Understand the concepts of acids and bases."
    ],
    "Physics": [
        "•   Focus on understanding the laws and theories.",
        "•   Practice solving numerical problems.",
        "•   Draw free-body diagrams for mechanics problems.",
        "•   Understand the relationship between different physical quantities.",
        "•   Review units and conversions."
    ],
    "Geography": [
        "•   Study maps regularly.",
        "•   Learn about different landforms and bodies of water.",
        "•   Understand climate zones and weather patterns.",
        "•   Study population distribution and cultural geography.",
        "•   Review major countries, capitals, and landmarks."
    ],
    "History": [
        "•   Create timelines of events.",
        "•   Understand the cause and effect of historical events.",
        "•   Read primary and secondary sources.",
        "•   Focus on key figures and their impact.",
        "•   Review different historical periods and their characteristics."
    ],
    "Commerce": [
        "•   Understand basic economic principles.",
        "•   Learn about different types of businesses.",
        "•   Study accounting basics and financial statements.",
        "•   Understand marketing and sales concepts.",
        "•   Review business laws and regulations."
    ],
    "Social Studies": [
        "•   Understand different forms of government.",
        "•   Learn about civics and citizenship.",
        "•   Study major social issues and their impacts.",
        "•   Review historical events related to social change.",
        "•   Understand cultural diversity and human rights."
    ]
}


def generate_study_tips_nlp(text):
    stop_words = set(stopwords.words('english'))
    words = word_tokenize(text.lower())
    keywords = [w for w in words if w.alnum() and w not in stop_words]
    top_keywords = keywords[:5]
    if not top_keywords:
        return "Please enter some text to generate study tips based on keywords."
    tips = "\n".join([f"• Revise '{kw}' regularly." for kw in top_keywords])
    return f"🧩 Keyword-based Study Tips:\n{tips}\n• Practice questions daily."

def get_subject_study_tips(subject):
    tips = subject_study_tips.get(subject, ["No specific tips available for this subject yet."])
    return "📚 General Study Tips:\n" + "\n".join(tips)


def feedback_generator(subject, score_percent):
    motivational = ""
    if score_percent >= 90:
        motivational = f"🌟 Excellent work on {subject}! Keep it up!"
    elif score_percent >= 70:
        motivational = f"💪 Good job on {subject}, you're improving!"
    elif score_percent >= 50:
        motivational = f"🙂 Nice try! Review some {subject} topics for better results."
    else:
        motivational = f"📘 Don’t worry! Keep practicing {subject} and you’ll get there!"

    return f"**{motivational}**\n\n*\"{get_random_quote()}\"*"

def generate_daily_study_plan_csv_and_chart(total_hours):
    subjects = list(quizzes.keys())
    hours_per_subject = round(total_hours / len(subjects), 2)
    plan = [{"Day": i+1, "Subject": s, "Hours": hours_per_subject} for i, s in enumerate(subjects)]
    df = pd.DataFrame(plan)

    # Create temporary file for CSV
    with tempfile.NamedTemporaryFile(delete=False, suffix=".csv") as tmp_csv:
        df.to_csv(tmp_csv.name, index=False)
        csv_filepath = tmp_csv.name

    # Pie chart image
    plt.figure(figsize=(6,4))
    plt.pie(df["Hours"], labels=df["Subject"], autopct="%1.1f%%")
    plt.title("Daily Study Plan Distribution")
    plt.tight_layout()

    # Create temporary file for image
    with tempfile.NamedTemporaryFile(delete=False, suffix=".png") as tmp_img:
        plt.savefig(tmp_img.name)
        img_filepath = tmp_img.name
    plt.close()

    return df, csv_filepath, img_filepath

def generate_weekly_study_plan_single_subject(subject, total_hours_per_week):
    days_of_week = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
    hours_per_day = round(total_hours_per_week / 7, 2)
    plan = [{"Day": day, "Subject": subject, "Hours": hours_per_day} for day in days_of_week]
    df = pd.DataFrame(plan)
    return df

def generate_weekly_study_plan_all_subjects(total_hours_per_week):
    subjects = list(quizzes.keys())
    days_of_week = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
    hours_per_subject_per_day = round(total_hours_per_week / len(subjects) / 7, 2)

    plan = []
    # Create a list of (day, subject) pairs, cycling through subjects for each day
    day_subject_pairs = [(day, subject) for day in days_of_week for subject in subjects]

    # Distribute hours
    for day, subject in day_subject_pairs:
         plan.append({"Day": day, "Subject": subject, "Hours": hours_per_subject_per_day})


    df = pd.DataFrame(plan)
    return df


# =========================
# GRADIO APP
# =========================
def start():
    with gr.Blocks(title="AI Study Pal", theme=gr.themes.Soft()) as demo:
        gr.Markdown("# 🎓 AI Study Pal - Intelligent Study Assistant")
        gr.Markdown("Select a feature tab below to get started.")

        with gr.Tab("📘 Take Quiz"):
            with gr.Row():
                subject_dd = gr.Dropdown(list(quizzes.keys()), label="Select Subject", value="English")
                start_btn = gr.Button("Start Quiz")

            # Pre-create 20 radio components (hidden initially)
            radios = []
            for i in range(20):
                r = gr.Radio(choices=[""], label=f"Q{i+1}.", visible=False, interactive=True)
                radios.append(r)

            submit_btn = gr.Button("Submit Answers", visible=False)
            result_box = gr.Markdown("")
            feedback_box = gr.Markdown("")
            # hidden state to store correct answers sequence and subject
            correct_state = gr.State([])
            quiz_subject_state = gr.State("") # State to store the subject of the current quiz

            # When Start Quiz clicked: populate 20 radios and store correct answers
            def start_quiz(subject):
                qlist = random.sample(quizzes[subject], 20)
                # Prepare updates for the 20 radios
                updates = []
                for i, (q, opts, ans) in enumerate(qlist):
                    updates.append(gr.update(label=f"Q{i+1}. {q}", choices=opts, value=None, visible=True))
                # if more outputs expected than updates, pad
                # store correct answers and subject in state
                correct_answers = [ans for (_, _, ans) in qlist]
                # show submit button
                submit_update = gr.update(visible=True)
                # reset result & feedback
                reset_result = ""
                reset_feedback = ""
                return (*updates, correct_answers, subject, submit_update, reset_result, reset_feedback)

            # Build outputs list: 20 radios, correct_state, quiz_subject_state, submit visibility, result, feedback
            outputs_for_start = [r for r in radios] + [correct_state, quiz_subject_state, submit_btn, result_box, feedback_box]
            start_btn.click(fn=start_quiz, inputs=[subject_dd], outputs=outputs_for_start)

            # Submit answers: compare radio values to correct_state
            def submit_answers(*args):
                # args: 20 radio values followed by correct_state (list) and quiz_subject_state (str)
                radio_vals = list(args[:20])
                correct_answers = args[20] if len(args) > 20 else []
                quiz_subject = args[21] if len(args) > 21 else "Unknown Subject" # Retrieve the quiz subject
                score = 0
                feedback_lines = []
                for i, user_ans in enumerate(radio_vals):
                    correct = correct_answers[i] if i < len(correct_answers) else None
                    if user_ans == correct:
                        score += 1
                        feedback_lines.append(f"Q{i+1}: ✅")
                    else:
                        correct_ans_text = correct if correct is not None else "N/A"
                        feedback_lines.append(f"Q{i+1}: ❌ (Correct: **{correct_ans_text}**)")

                percent = round(score / 20 * 100, 1)
                result_md = f"**Score:** {score}/20  —  **{percent}%**"
                motivational = feedback_generator(quiz_subject, percent) # Pass the actual quiz subject
                details_md = "<br>".join(feedback_lines)
                return result_md + "<br><br>" + details_md, motivational

            # connect submit_btn to evaluation. Pass radios' values + correct_state + quiz_subject_state as inputs
            submit_btn.click(fn=submit_answers, inputs=[*radios, correct_state, quiz_subject_state], outputs=[result_box, feedback_box])

        with gr.Tab("🧠 Daily Study Plan"):
            gr.Markdown("Generate a daily study plan. The plan is evenly split across all subjects.")
            hours_slider = gr.Slider(1, 24, value=6, step=1, label="Total study hours per day")
            plan_btn = gr.Button("Generate Daily Study Plan")
            plan_table = gr.Dataframe(headers=["Day", "Subject", "Hours"])
            download_csv = gr.File(label="Download daily_study_plan.csv")
            plan_chart = gr.Image(label="Daily study distribution chart")

            def make_daily_plan(hours):
                df, csv_filepath, img_filepath = generate_daily_study_plan_csv_and_chart(hours)
                # return dataframe, file, image filename
                return df, csv_filepath, img_filepath

            plan_btn.click(fn=make_daily_plan, inputs=hours_slider, outputs=[plan_table, download_csv, plan_chart])

        with gr.Tab("📅 Weekly Study Plan (Single Subject)"):
            gr.Markdown("Generate a weekly study plan for a specific subject. The plan is evenly split across 7 days.")
            weekly_subject_dd = gr.Dropdown(list(quizzes.keys()), label="Select Subject", value="English")
            weekly_hours_slider = gr.Slider(1, 168, value=20, step=1, label="Total study hours per week for this subject")
            weekly_plan_btn = gr.Button("Generate Weekly Study Plan")
            weekly_plan_table = gr.Dataframe(headers=["Day", "Subject", "Hours"])

            weekly_plan_btn.click(
                fn=generate_weekly_study_plan_single_subject,
                inputs=[weekly_subject_dd, weekly_hours_slider],
                outputs=weekly_plan_table
            )

        with gr.Tab("📅 Weekly Study Plan (All Subjects)"):
            gr.Markdown("Generate a weekly study plan distributing hours across all subjects.")
            weekly_all_hours_slider = gr.Slider(1, 168, value=40, step=1, label="Total study hours per week (All Subjects)")
            weekly_all_plan_btn = gr.Button("Generate Weekly Plan (All Subjects)")
            weekly_all_plan_table = gr.Dataframe(headers=["Day", "Subject", "Hours"])

            weekly_all_plan_btn.click(
                fn=generate_weekly_study_plan_all_subjects,
                inputs=[weekly_all_hours_slider],
                outputs=weekly_all_plan_table
            )


        with gr.Tab("💡 Study Tips"):

            gr.Markdown("Select a subject to see general study tips.")
            subject_tips_dd = gr.Dropdown(list(quizzes.keys()), label="Select Subject for General Tips", value="English")
            subject_tips_btn = gr.Button("Show General Study Tips")
            tips_out_subject = gr.Textbox(label="General Study Tips", interactive=False)

            gr.Markdown("---") # Separator


            subject_tips_btn.click(fn=get_subject_study_tips, inputs=subject_tips_dd, outputs=tips_out_subject)


        with gr.Tab("🌐 Resources"):
            res_sub = gr.Dropdown(list(quizzes.keys()), label="Select Subject", value="English")
            res_btn = gr.Button("Show Resource")
            res_out = gr.Markdown("")
            def show_resource(subject):
                # Check if the resource is a list or a single URL
                resource_entry = resources.get(subject)
                if isinstance(resource_entry, list):
                    # If it's a list, format each URL as a link
                    markdown_output = f"🌐 **{subject} Resources:**<br>"
                    for url in resource_entry:
                        # Attempt to create a more user-friendly link name if possible, otherwise use the URL
                        link_name = url # Default to URL
                        if "khanacademy.org" in url:
                            link_name = "Khan Academy"
                        elif "khanglobalstudies.com" in url:
                            link_name = "Khan Global Studies"
                        elif "library.sjsu.edu" in url:
                            link_name = "SJSU Library"
                        elif "textkit.com" in url:
                             link_name = "Textkit"
                        elif "wikipedia.org" in url:
                             link_name = "Wikipedia"
                        elif "uiowa.edu" in url: # Added for the new Greek resource
                            link_name = "University of Iowa" # More general name

                        markdown_output += f"- [{link_name}]({url})<br>"
                    return markdown_output
                elif isinstance(resource_entry, str):
                    # If it's a single URL, format it as a single link
                    link_name = resource_entry
                    if "khanacademy.org" in resource_entry:
                        link_name = "Khan Academy"
                    elif "geeksfor geeks.org" in resource_entry:
                        link_name = "GeeksforGeeks"
                    elif "wikipedia.org" in resource_entry:
                        link_name = "Wikipedia"
                    return f"🌐 **{subject} Resource:** [{link_name}]({resource_entry})"
                else:
                    return "No resource found for this subject."

            res_btn.click(fn=show_resource, inputs=res_sub, outputs=res_out)

        gr.Markdown("---")
        gr.Markdown("Created with ❤️ — AI Study Pal (Gradio).")

    demo.launch()

if __name__ == "__main__":
    start()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://3f93d67b9cc73e8f93.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
